# Data Preprocessing

## Import Libraries

In [2]:
import pandas as pd
import missingno as msno
import numpy as np
from matplotlib import pyplot as plt

## Import Dataset

In [3]:
users = pd.read_csv('./CSVs/BX-Users.csv', error_bad_lines=False, sep=';', encoding="latin1")

In [4]:
books = pd.read_csv('./CSVs/BX-Books.csv', error_bad_lines=False, sep=';', encoding="latin1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
c:\users\asreno\appdata\local\programs\python\python39\lib\site-packages\I

In [5]:
ratings = pd.read_csv('./CSVs/BX-Book-Ratings.csv', error_bad_lines=False, sep=';', encoding="latin1")

In [ ]:
print('The shape of Ratings is: ' + str(ratings.shape))
print('The shape of Users is: ' + str(users.shape))
print('The shape of Books is: ' + str(books.shape))

## Data Preprocessing

### Users DataFrame

### Books DataFrame

### Ratings DataFrame